In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt

data_path = 'data_path'
df = pd.read_csv(data_path)

In [ ]:
#Subset the data for training
df = df.drop([insert columns after 1st trimester], axis=1)
df = df.drop_duplicates()

# General Data Preparation

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

full_df = df
# 'GDM' is the target variable
target_column = 'GDM'

# Identify categorical and numerical features
categorical_features = full_df.select_dtypes(include=['object']).columns.tolist()
numerical_features = full_df.select_dtypes(exclude=['object']).columns.tolist()
numerical_features.remove(target_column)
numerical_features.remove('ID')

# Preprocessor setup
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'  # Include columns not specified in transformers unchanged
)

# Apply preprocessing to the entire dataset except 'ID' and 'GDM'
features = full_df.drop([target_column, 'ID'], axis=1)
X_processed = preprocessor.fit_transform(features)
y = full_df[target_column].values

# Convert processed features back to DataFrame with transformed column names
columns_transformed = (list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
                       + numerical_features)
X_processed_df = pd.DataFrame(X_processed, columns=columns_transformed)

# Reattach 'ID' to processed features for grouping in splits
X_processed_df['ID'] = full_df['ID'].values

# Initial group-based split
# This is to ensure that all ID are in the same set
gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
train_idxs, temp_idxs = next(gss.split(X_processed_df, groups=X_processed_df['ID']))
train_set = X_processed_df.iloc[train_idxs]
y_train = y[train_idxs]
temp_set = X_processed_df.iloc[temp_idxs]
y_temp = y[temp_idxs]

# Split temp set into validation and test sets
gss_temp = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=42)
val_idxs, test_idxs = next(gss_temp.split(temp_set, groups=temp_set['ID']))
validation_set = temp_set.iloc[val_idxs]
y_val = y_temp[val_idxs]
test_set = temp_set.iloc[test_idxs]
y_test = y_temp[test_idxs]

# Separate features from 'ID' for training, validation, and test sets
X_train = train_set.drop('ID', axis=1)
X_val = validation_set.drop('ID', axis=1)
X_test = test_set.drop('ID', axis=1)

# General Modelling

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, average_precision_score,
    confusion_matrix, f1_score, brier_score_loss
)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.calibration import calibration_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample


# Assuming X_train, X_val, y_train, y_val are already defined
# Define the parameter distributions for the RandomizedSearchCV for each model
param_distributions = {
    'RandomForestClassifier': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    },
    'LogisticRegression': {
        'C': np.logspace(-4, 4, 20),
        'solver': ['liblinear', 'lbfgs']
    },
    'XGBClassifier': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5, 10]
    },
    'ExplainableBoostingClassifier': {
        'learning_rate': [0.01, 0.1, 1],
        'max_leaves': [3, 10, 20]
    }
}

models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'XGBClassifier': XGBClassifier(eval_metric='logloss'),
    'ExplainableBoostingClassifier': ExplainableBoostingClassifier()
}

best_estimators = {}
roc_results = {}
ap_results = {}
metrics_results = {}  # Dictionary to store all the additional metrics
calibration_data = {}

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def bootstrap_auc_ci(y_true, y_score, n_bootstraps=1000, ci=0.95, seed=42):
    rng = np.random.RandomState(seed)
    bootstrapped_scores = []

    for _ in range(n_bootstraps):
        indices = rng.choice(range(len(y_score)), size=len(y_score), replace=True)
        if len(np.unique(y_true[indices])) < 2:
            continue
        score = roc_auc_score(y_true[indices], y_score[indices])
        bootstrapped_scores.append(score)

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    lower = sorted_scores[int((1.0 - ci) / 2 * len(sorted_scores))]
    upper = sorted_scores[int((1.0 + ci) / 2 * len(sorted_scores))]
    return lower, upper
    
for name, model in models.items():
    random_search = RandomizedSearchCV(model, param_distributions=param_distributions[name], n_iter=10, scoring='roc_auc', n_jobs=-1, 
                                       cv=stratified_kfold, verbose=1, random_state=42)
    random_search.fit(X_train, y_train)
    best_estimators[name] = random_search.best_estimator_
    
    # Predict probabilities for the validation set
    y_score = best_estimators[name].predict_proba(X_val)[:, 1]
    
    # Compute ROC AUC
    fpr, tpr, _ = roc_curve(y_val, y_score)
    roc_auc = auc(fpr, tpr)
    auc_ci_lower, auc_ci_upper = bootstrap_auc_ci(np.array(y_val), np.array(y_score))
    roc_results[name] = (fpr, tpr, roc_auc, auc_ci_lower, auc_ci_upper)
    
    # Compute Average Precision
    precision, recall, _ = precision_recall_curve(y_val, y_score)
    ap_score = average_precision_score(y_val, y_score)
    ap_results[name] = (precision, recall, ap_score)

    # Convert probabilities to binary predictions using 0.5 threshold
    y_pred = (y_score >= 0.5).astype(int)
    
    # Compute Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    
    # Compute Sensitivity (Recall for Positive Class)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # Compute Specificity
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # Compute F1 Score
    f1 = f1_score(y_val, y_pred)
    
    # Compute Brier Score
    brier = brier_score_loss(y_val, y_score)
    
    # Store the computed metrics
    metrics_results[name] = {
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'F1 Score': f1,
        'Brier Score': brier
    }
    
    # Calibration slope & intercept
    calib_lr = LogisticRegression(solver='lbfgs')
    eps = 1e-15
    y_score_clipped = np.clip(y_score, eps, 1 - eps)
    logit_y_score = np.log(y_score_clipped / (1 - y_score_clipped))
    calib_lr.fit(logit_y_score.reshape(-1, 1), y_val)
    calib_intercept = calib_lr.intercept_[0]
    calib_slope = calib_lr.coef_[0][0]

    # O:E ratio
    expected_positives = np.sum(y_score)
    observed_positives = np.sum(y_val)
    oe_ratio = observed_positives / expected_positives if expected_positives != 0 else np.nan

    # Update calibration_data
    calibration_data[name] = {
        'y_true': y_val,
        'y_pred_prob': y_score,
        'Calibration Intercept': calib_intercept,
        'Calibration Slope': calib_slope,
        'O:E Ratio': oe_ratio
    }

# After training all models, you can print or log the results
for name in models.keys():
    print(f"\nModel: {name}")
    print(f"Best Estimator: {best_estimators[name]}")
    print(f"ROC AUC: {roc_results[name][2]:.3f}")
    print(f"Average Precision (AP): {ap_results[name][2]:.3f}")
    print(f"Sensitivity: {metrics_results[name]['Sensitivity']:.3f}")
    print(f"Specificity: {metrics_results[name]['Specificity']:.3f}")
    print(f"F1 Score: {metrics_results[name]['F1 Score']:.3f}")
    print(f"Brier Score: {metrics_results[name]['Brier Score']:.3f}")
    print(f"ROC AUC: {roc_results[name][2]:.3f} (95% CI: {roc_results[name][3]:.3f} – {roc_results[name][4]:.3f})")
    print(f"Calibration Slope: {calibration_data[name]['Calibration Slope']:.3f}")
    print(f"Calibration Intercept: {calibration_data[name]['Calibration Intercept']:.3f}")
    print(f"O:E Ratio: {calibration_data[name]['O:E Ratio']:.3f}")

# Save the results using joblib
from joblib import dump
results = {
    'best_estimators': best_estimators,
    'roc_results': roc_results,
    'ap_results': ap_results,
    'metrics_results': metrics_results,
    'calibration_data': calibration_data
}
dump(results, 'model_results.joblib')
print("Model results saved to 'model_results.joblib'.")
    
plt.rcParams.update({'font.size': 14, 'figure.dpi': 100})

# Plot ROC curves
plt.figure(figsize=(10, 10))
for name, (fpr, tpr, roc_auc, auc_ci_lower, auc_ci_upper) in roc_results.items():
    plt.plot(fpr, tpr, label=f'{name} (area = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Plot Precision-Recall curves
plt.figure(figsize=(10, 10))
for name, (precision, recall, ap_score) in ap_results.items():
    plt.plot(recall, precision, label=f'{name} (AP = {ap_score:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend(loc="lower left")
plt.show()

## Repeat above steps for Nulliparous Model and FTP-9

## FTP-9

In [ ]:
sub_df = df[['ID','Ethnic Origin of Patient','Age at booking', 'Hx_GDM', 'BMI', 'FH Diabetes',
             'Other Endocrine probs', 'Systolic BP at booking', 'Diastolic BP at booking', 'Parity (not inc.multiple)',
             'GDM']]

## Nulliparous

In [ ]:
first_df = df[df['Parity (not inc.multiple)'] == 0]

# Sequential Generation

In [ ]:
import pandas as pd

# Convert 'Date of Birth' to datetime
df['Date of Birth'] = pd.to_datetime(df['Date of Birth'])

# Sort by 'ID' and 'Date of Birth' so that they are together
df = df.sort_values(by=['ID', 'Date of Birth'])

# Create a shifted column to get the future pregnancy 'GDM' status.
# This shifts the 'GDM' values by one row upwards within each group.
df['future_GDM'] = df.groupby('ID')['GDM'].shift(-1)

# Drop rows where there is no future pregnancy data
df = df.dropna(subset=['future_GDM'])

## Data Prep

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

# 'GDM' is the target variable
target_column = 'future_GDM'

# Identify categorical and numerical features
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
numerical_features = df.select_dtypes(exclude=['object']).columns.tolist()
numerical_features.remove(target_column)
numerical_features.remove('ID')

# Preprocessor setup
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'  # Include columns not specified in transformers unchanged
)

# Apply preprocessing to the entire dataset except 'ID' and 'GDM'
features = df.drop([target_column, 'ID'], axis=1)
X_processed = preprocessor.fit_transform(features)
y = df[target_column].values

# Convert processed features back to DataFrame with transformed column names
columns_transformed = (list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
                       + numerical_features)
X_processed_df = pd.DataFrame(X_processed, columns=columns_transformed)

# Reattach 'ID' to processed features for grouping in splits
X_processed_df['ID'] = df['ID'].values

# Initial group-based split
# This is to ensure that all patients are in the same set
gss = GroupShuffleSplit(n_splits=1, train_size=0.7, random_state=42)
train_idxs, temp_idxs = next(gss.split(X_processed_df, groups=X_processed_df['ID']))
train_set = X_processed_df.iloc[train_idxs]
y_train = y[train_idxs]
temp_set = X_processed_df.iloc[temp_idxs]
y_temp = y[temp_idxs]

# Split temp set into validation and test sets
gss_temp = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=42)
val_idxs, test_idxs = next(gss_temp.split(temp_set, groups=temp_set['ID']))
validation_set = temp_set.iloc[val_idxs]
y_val = y_temp[val_idxs]
test_set = temp_set.iloc[test_idxs]
y_test = y_temp[test_idxs]

# Separate features from 'ID' for training, validation, and test sets
X_train = train_set.drop('ID', axis=1)
X_val = validation_set.drop('ID', axis=1)
X_test = test_set.drop('ID', axis=1)

## Modelling

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, average_precision_score,
    confusion_matrix, f1_score, brier_score_loss, roc_auc_score
)
import matplotlib.pyplot as plt
import numpy as np

# Assuming X_train, X_val, y_train, y_val are already defined

# Define the parameter distributions for the RandomizedSearchCV for each model
param_distributions = {
    'RandomForestClassifier': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    },
    'LogisticRegression': {
        'C': np.logspace(-4, 4, 20),
        'solver': ['liblinear', 'lbfgs']
    },
    'XGBClassifier': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5, 10]
    },
    'ExplainableBoostingClassifier': {
        'learning_rate': [0.01, 0.1, 1],
        'max_leaves': [3, 10, 20]
    }
}

models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'XGBClassifier': XGBClassifier(eval_metric='logloss'),
    'ExplainableBoostingClassifier': ExplainableBoostingClassifier()
}

best_estimators = {}
roc_results = {}
ap_results = {}
metrics_results = {}  # Dictionary to store all the additional metrics
calibration_data = {}

# Define stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define a bootstrap function to compute the 95% CI for the AUC
def bootstrap_auc_ci(y_true, y_score, n_bootstraps=1000, ci=0.95, seed=42):
    rng = np.random.RandomState(seed)
    bootstrapped_scores = []
    for _ in range(n_bootstraps):
        indices = rng.choice(range(len(y_score)), size=len(y_score), replace=True)
        if len(np.unique(y_true[indices])) < 2:
            continue
        score = roc_auc_score(y_true[indices], y_score[indices])
        bootstrapped_scores.append(score)
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    lower = sorted_scores[int((1.0 - ci) / 2 * len(sorted_scores))]
    upper = sorted_scores[int((1.0 + ci) / 2 * len(sorted_scores))]
    return lower, upper

for name, model in models.items():
    random_search = RandomizedSearchCV(
        model, 
        param_distributions=param_distributions[name], 
        n_iter=10, 
        scoring='roc_auc', 
        n_jobs=-1, 
        cv=skf, 
        verbose=1, 
        random_state=42
    )
    random_search.fit(X_train, y_train)
    best_estimators[name] = random_search.best_estimator_
    
    # Predict probabilities for the validation set
    y_score = best_estimators[name].predict_proba(X_val)[:, 1]
    
    # --- Calibration Metrics ---
    # Calibration slope & intercept
    calib_lr = LogisticRegression(solver='lbfgs')
    eps = 1e-15
    y_score_clipped = np.clip(y_score, eps, 1 - eps)
    logit_y_score = np.log(y_score_clipped / (1 - y_score_clipped))
    calib_lr.fit(logit_y_score.reshape(-1, 1), y_val)
    calib_intercept = calib_lr.intercept_[0]
    calib_slope = calib_lr.coef_[0][0]
    
    expected_positives = np.sum(y_score)
    observed_positives = np.sum(y_val)
    oe_ratio = observed_positives / expected_positives if expected_positives != 0 else np.nan
    
    calibration_data[name] = {
        'y_true': y_val,
        'y_pred_prob': y_score,
        'Calibration Intercept': calib_intercept,
        'Calibration Slope': calib_slope,
        'O:E Ratio': oe_ratio
    }
    
    # --- ROC and Precision-Recall Metrics ---
    fpr, tpr, _ = roc_curve(y_val, y_score)
    roc_auc = auc(fpr, tpr)
    auc_ci_lower, auc_ci_upper = bootstrap_auc_ci(np.array(y_val), np.array(y_score))
    roc_results[name] = (fpr, tpr, roc_auc, auc_ci_lower, auc_ci_upper)
    
    precision, recall, _ = precision_recall_curve(y_val, y_score)
    ap_score = average_precision_score(y_val, y_score)
    ap_results[name] = (precision, recall, ap_score)
    
    # Convert probabilities to binary predictions using a 0.5 threshold
    y_pred = (y_score >= 0.5).astype(int)
    
    # Compute Confusion Matrix and additional metrics
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = f1_score(y_val, y_pred)
    brier = brier_score_loss(y_val, y_score)
    
    metrics_results[name] = {
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'F1 Score': f1,
        'Brier Score': brier
    }
    
# After training all models, print the results
for name in models.keys():
    print(f"\nModel: {name}")
    print(f"Best Estimator: {best_estimators[name]}")
    print(f"ROC AUC: {roc_results[name][2]:.3f} (95% CI: {roc_results[name][3]:.3f} – {roc_results[name][4]:.3f})")
    print(f"Average Precision (AP): {ap_results[name][2]:.3f}")
    print(f"Sensitivity: {metrics_results[name]['Sensitivity']:.3f}")
    print(f"Specificity: {metrics_results[name]['Specificity']:.3f}")
    print(f"F1 Score: {metrics_results[name]['F1 Score']:.3f}")
    print(f"Brier Score: {metrics_results[name]['Brier Score']:.3f}")
    print(f"Calibration Slope: {calibration_data[name]['Calibration Slope']:.3f}")
    print(f"Calibration Intercept: {calibration_data[name]['Calibration Intercept']:.3f}")
    print(f"O:E Ratio: {calibration_data[name]['O:E Ratio']:.3f}")

# Save the results using joblib
from joblib import dump
results = {
    'best_estimators': best_estimators,
    'roc_results': roc_results,
    'ap_results': ap_results,
    'metrics_results': metrics_results,
    'calibration_data': calibration_data
}
dump(results, 'prev_model_results.joblib')
print("Model results saved to 'prev_model_results.joblib'.")

plt.rcParams.update({'font.size': 14, 'figure.dpi': 100})

# Plot ROC curves with confidence intervals in the legend
plt.figure(figsize=(10, 10))
for name, (fpr, tpr, roc_auc, auc_ci_lower, auc_ci_upper) in roc_results.items():
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f}, 95% CI = [{auc_ci_lower:.3f}, {auc_ci_upper:.3f}])')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Plot Precision-Recall curves
plt.figure(figsize=(10, 10))
for name, (precision, recall, ap_score) in ap_results.items():
    plt.plot(recall, precision, label=f'{name} (AP = {ap_score:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend(loc="lower left")
plt.show()